In [2]:
pip install charset-normalizer==2.1.0

  Attempting uninstall: charset-normalizer
    Found existing installation: charset-normalizer 3.1.0
    Uninstalling charset-normalizer-3.1.0:
      Successfully uninstalled charset-normalizer-3.1.0
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\silvi\\anaconda3\\envs\\test\\Lib\\site-packages\\~harset_normalizer\\md.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import f1_score
import tensorflow as tf

# Set the random seed for reproducibility
tf.random.set_seed(42)

# Load the training data
train_data = pd.read_csv("train.csv")

# Preprocess the text data using Tokenizer
MAX_NB_WORDS = 50000
MAX_SEQ_LENGTH = 250
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(train_data["text"])
sequences = tokenizer.texts_to_sequences(train_data["text"])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
train_feature = pad_sequences(sequences, maxlen=MAX_SEQ_LENGTH)

# Define the RNN model
EMBEDDING_DIM = 100
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=MAX_SEQ_LENGTH))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# Convert the labels to one-hot encoding
train_labels = pd.get_dummies(train_data["label"]).values

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_feature, train_labels, test_size=0.2, random_state=42)

# Train the model on a GPU
with tf.device('/GPU:0'):
    checkpoint = ModelCheckpoint("best_model.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    earlystop = EarlyStopping(monitor='val_accuracy', patience=3, verbose=1, mode='max')
    callbacks_list = [checkpoint, earlystop]
    history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val), callbacks=callbacks_list)

# Evaluate the model on the validation set using macro F1-score
y_val_pred = model.predict(X_val)
y_val_pred_labels = y_val_pred.argmax(axis=1)
y_val_labels = y_val.argmax(axis=1)
f1_macro = f1_score(y_val_labels, y_val_pred_labels, average='macro')
print("Validation set macro F1-score: {:.4f}".format(f1_macro))

# Load the test data
test_data = pd.read_csv("test.csv")

# Preprocess the test data using Tokenizer
test_sequences = tokenizer.texts_to_sequences(test_data["text"])
test_feature = pad_sequences(test_sequences, maxlen=MAX_SEQ_LENGTH)

# Predict the labels for the test data
y_test_pred = model.predict(test_feature)
y_test_pred_labels = y_test_pred.argmax(axis=1)

# Create a submission file
submission = pd.DataFrame({"id": test_data["id"], "label": y_test_pred_labels})
submission.to_csv("submission.csv", index=False)


Found 298918 unique tokens.
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 250, 100)          5000000   
                                                                 
 lstm_1 (LSTM)               (None, 128)               117248    
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 8)                 520       
                                                                 
Total params: 5,126,024
Trainable params: 5,126,024
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
